In [570]:
import tensorflow as tf
import pandas as pd

print(tf.__version__)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

2.16.1


In [571]:
dataset = pd.read_excel('../data/pisos.xlsx', index_col='ID')

In [572]:
dataset.head(5)

,Planta,Metros cuadrados totales,Metros cuadrados habitables,Numero habitaciones,Numero baños,Estado piso,Amueblado,Zona,Tipo piso,Orientación,Terraza,Aire acondicionado,Urbanización,Piscina en la urbanización,Ascensor,Garaje,Años antigüedad piso,Servicios incluidos,Accesibilidad,Años contrato alquiler,Precio alquiler al año,Gastos generales al año,Gastos iniciales,Beneficios totales
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,7,87.5,79.4,2,1.0,nuevo,True,Iturrama,piso,NE,True,True,False,False,True,False,1,luz.agua,4,9,14060,1787,220,3196.814
2,3,71.2,64.3,2,1.5,reformado,True,Ensanche,piso,SE,False,True,True,False,True,True,7,agua.comunidad,5,5,10525,1124,572,6227.500
3,5,104.6,90.3,3,2.0,buen estado,True,Mendebaldea,piso,W,True,False,False,False,True,True,15,agua.luz.comunidad,3,4,18555,1512,2810,4160.112
4,13,148.1,143.3,5,2.0,regular estado,True,Mendebaldea,atico,NW,True,True,True,False,True,True,30,luz.agua.comunidad,2,10,24430,2625,7630,20183.590
5,5,80.8,69.0,2,1.0,buen estado,False,Rochapea,piso,E,False,False,False,False,False,True,10,luz.comunidad,4,10,10565,1134,16440,-8424.920


In [573]:
dataset.shape

(180, 24)

---
---
En primer lugar añadimos una nueva columna al final del dataset del precio al que se alquila el piso siguiendo el siguiente criterio:

'Precio alquiler aplicado al año' = ('Beneficios' + 'Precio alquiler al año' * 'Años contrato alquiler' + 
'Gastos generales al año' * 'Años contrato alquiler' + 'Gastos iniciales') / 'Años contrato alquiler'

Realizamos esto en primer lugar para poder representar graficamente la relación de algunas de las variables numéricas que consideremos importantes. 

Este 'Precio aplicado de alquiler al año' va a ser el que va a intentar predecir nuestra red neuronal

In [574]:
def calc_precio_aplicado_año(row):
    alq = row['Precio alquiler al año']
    gast = row['Gastos generales al año']
    gast_o = row['Gastos iniciales']
    anos = row['Años contrato alquiler']
    beneficios = row['Beneficios totales']

    return (beneficios + alq * anos + gast * anos + gast_o) / anos

In [575]:
dataset['Precio alquiler aplicado al año'] = dataset.apply(calc_precio_aplicado_año, axis=1)

In [576]:
dataset['Precio alquiler aplicado al año'].head()

ID
1    16226.646
2    13008.900
3    21809.528
4    29836.359
5    12500.508
Name: Precio alquiler aplicado al año, dtype: float64

---
---
# 1) Vamos a ir viendo el tipo de cada característica y transformandola para una mejor interpretación

In [577]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 1 to 180
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Planta                           180 non-null    int64  
 1   Metros cuadrados totales         180 non-null    float64
 2   Metros cuadrados habitables      180 non-null    float64
 3   Numero habitaciones              180 non-null    int64  
 4   Numero baños                     180 non-null    float64
 5   Estado piso                      180 non-null    object 
 6   Amueblado                        180 non-null    bool   
 7   Zona                             180 non-null    object 
 8   Tipo piso                        180 non-null    object 
 9   Orientación                      180 non-null    object 
 10  Terraza                          180 non-null    bool   
 11  Aire acondicionado               180 non-null    bool   
 12  Urbanización               

0) ID: int

Lo usamos como ID del DataFrame

1) Planta: int

In [578]:
dataset['Planta'].value_counts()

Planta
3     32
1     28
2     24
7     18
6     16
4     16
8     14
0     11
5      9
12     5
9      2
10     2
11     2
13     1
Name: count, dtype: int64

2) Metros cuadrados totales: float

In [579]:
print(dataset['Metros cuadrados totales'].min())
print(dataset['Metros cuadrados totales'].max())

71.2
205.0


3) Metros cuadrados habitables: float

In [580]:
print(dataset['Metros cuadrados habitables'].min())
print(dataset['Metros cuadrados habitables'].max())

64.3
184.9


4) Numero habitaciones: int

In [581]:
dataset['Numero habitaciones'].value_counts()

Numero habitaciones
3    60
4    44
2    40
5    36
Name: count, dtype: int64

5) Numero baños (puntúa 0.5 un baño sin ducha): float

In [582]:
dataset['Numero baños'].value_counts()

Numero baños
2.0    74
1.0    39
1.5    34
3.0    17
2.5    16
Name: count, dtype: int64

6) Estado piso: {'nuevo', 'reformado', 'buen estado', 'regular estado'}

Tenemos dos opciones:
    
a) Transformamos cada categoría en un entero, en nuestro caso ordenado de mejor a peor estado siendo: 
    
    nuevo: 4
    reformado: 3
    buen estado: 2
    regular estado: 1

b) Aplicamos one-hot encoding añadiendo 4 nuevas columnas, una por cada categoría, y escribiendo un 1 cuando coincida la categoría y 0 en caso contrario. Además hay que eliminar la columna 'Estado piso'

-> Empezaremos aplicando la primera opción usando la función map

In [583]:
dataset['Estado piso'].value_counts()

Estado piso
buen estado       88
regular estado    41
reformado         31
nuevo             20
Name: count, dtype: int64

In [584]:
def puntuar_estado_piso(x):
    if x == 'nuevo':
        return 4
    elif x == 'reformado':
        return 3
    elif x == 'buen estado':
        return 2
    else:
        return 1

In [585]:
dataset['Estado piso'] = dataset['Estado piso'].map(puntuar_estado_piso)

In [586]:
dataset['Estado piso'].value_counts()

Estado piso
2    88
1    41
3    31
4    20
Name: count, dtype: int64

7) Amueblado: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [587]:
dataset['Amueblado'].value_counts()

Amueblado
True     137
False     43
Name: count, dtype: int64

In [588]:
def codif_boolean(x):
    if x == True:
        return 1
    else:
        return 0

In [589]:
dataset['Amueblado'] = dataset['Amueblado'].map(codif_boolean)

In [590]:
dataset['Amueblado'].value_counts()

Amueblado
1    137
0     43
Name: count, dtype: int64

In [591]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 1 to 180
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Planta                           180 non-null    int64  
 1   Metros cuadrados totales         180 non-null    float64
 2   Metros cuadrados habitables      180 non-null    float64
 3   Numero habitaciones              180 non-null    int64  
 4   Numero baños                     180 non-null    float64
 5   Estado piso                      180 non-null    int64  
 6   Amueblado                        180 non-null    int64  
 7   Zona                             180 non-null    object 
 8   Tipo piso                        180 non-null    object 
 9   Orientación                      180 non-null    object 
 10  Terraza                          180 non-null    bool   
 11  Aire acondicionado               180 non-null    bool   
 12  Urbanización               

---
8) Zona: {'Casco viejo', 'Iturrama', 'Ensanche', 'Mendebaldea', 'Rochapea'}

Como en nuestro caso no tenemos información tan clara para concluir que zona es mejor a la otra numericamente vamos a aplicar one-hot enconding

In [592]:
dataset['Zona'].value_counts()

Zona
Mendebaldea    45
Iturrama       40
Ensanche       38
Casco viejo    29
Rochapea       28
Name: count, dtype: int64

In [593]:
one_hot_encoded = pd.get_dummies(dataset['Zona'])

zona_dataset = one_hot_encoded.map(codif_boolean)

dataset = pd.concat([dataset, zona_dataset], axis=1)

dataset = dataset.drop('Zona', axis=1)

In [594]:
print(dataset[['Casco viejo', 'Iturrama', 'Ensanche', 'Mendebaldea', 'Rochapea']].value_counts())

Casco viejo  Iturrama  Ensanche  Mendebaldea  Rochapea
0            0         0         1            0           45
             1         0         0            0           40
             0         1         0            0           38
1            0         0         0            0           29
0            0         0         0            1           28
Name: count, dtype: int64


---
9) Tipo piso: {'piso', 'atico'}

Como distinguimos entre dos tipos de piso usaremos una sola columna con los siguientes valores:

    1: 'piso'
    0: 'atico'

In [595]:
dataset['Tipo piso'].value_counts()

Tipo piso
piso     172
atico      8
Name: count, dtype: int64

In [596]:
def piso_atico(x):
    if x == 'piso':
        return 1
    else:
        return 0

In [597]:
dataset['Piso'] = dataset['Tipo piso'].map(piso_atico)
dataset = dataset.drop('Tipo piso', axis=1)

In [598]:
dataset['Piso'].value_counts()

Piso
1    172
0      8
Name: count, dtype: int64

---
10) Orientación: uno o dos de los siguientes valores: {'N', 'S', 'E', 'W'}

In [599]:
dataset['Orientación'].value_counts()

Orientación
E     54
N     26
NE    22
SE    22
SW    20
W     18
NW    17
S      1
Name: count, dtype: int64

In [600]:
opciones = ['N', 'S', 'E', 'W']

for opc in opciones:
    dataset[opc] = dataset['Orientación'].map(lambda x: 1 if opc in x else 0)

In [601]:
dataset = dataset.drop('Orientación', axis=1)

In [602]:
dataset[['N', 'S', 'E', 'W']].value_counts()

N  S  E  W
0  0  1  0    54
1  0  0  0    26
0  1  1  0    22
1  0  1  0    22
0  1  0  1    20
   0  0  1    18
1  0  0  1    17
0  1  0  0     1
Name: count, dtype: int64

---
11) Terraza: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [603]:
dataset['Terraza'].value_counts()

Terraza
False    98
True     82
Name: count, dtype: int64

In [604]:
dataset['Terraza'] = dataset['Terraza'].map(codif_boolean)

In [605]:
dataset['Terraza'].value_counts()

Terraza
0    98
1    82
Name: count, dtype: int64

---
12) Aire acondicionado: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [606]:
dataset['Aire acondicionado'].value_counts()

Aire acondicionado
True     109
False     71
Name: count, dtype: int64

In [607]:
dataset['Aire acondicionado'] = dataset['Aire acondicionado'].map(codif_boolean)

In [608]:
dataset['Aire acondicionado'].value_counts()

Aire acondicionado
1    109
0     71
Name: count, dtype: int64

---
13) Urbanización: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [609]:
dataset['Urbanización'].value_counts()

Urbanización
False    123
True      57
Name: count, dtype: int64

In [610]:
dataset['Urbanización'] = dataset['Urbanización'].map(codif_boolean)

In [611]:
dataset['Urbanización'].value_counts()

Urbanización
0    123
1     57
Name: count, dtype: int64

---
14) Piscina: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [612]:
dataset['Piscina en la urbanización'].value_counts()

Piscina en la urbanización
False    160
True      20
Name: count, dtype: int64

In [613]:
dataset['Piscina en la urbanización'] = dataset['Piscina en la urbanización'].map(codif_boolean)

In [614]:
dataset['Piscina en la urbanización'].value_counts()

Piscina en la urbanización
0    160
1     20
Name: count, dtype: int64

---
15) Ascensor: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [615]:
dataset['Ascensor'].value_counts()

Ascensor
True     147
False     33
Name: count, dtype: int64

In [616]:
dataset['Ascensor'] = dataset['Ascensor'].map(codif_boolean)

In [617]:
dataset['Ascensor'].value_counts()

Ascensor
1    147
0     33
Name: count, dtype: int64

---
16) Garaje: boolean

Transformamos el boolean con el siguiente criterio para un mejor manejo de los valores al ser numéricos:

    True: 1
    False: 0

In [618]:
dataset['Garaje'].value_counts()

Garaje
False    112
True      68
Name: count, dtype: int64

In [619]:
dataset['Garaje'] = dataset['Garaje'].map(codif_boolean)

In [620]:
dataset['Garaje'].value_counts()

Garaje
0    112
1     68
Name: count, dtype: int64

---
17) Años antigüedad piso: int

In [621]:
dataset['Años antigüedad piso'].value_counts()

Años antigüedad piso
8     34
3     31
10    20
4     20
6     18
9     17
12    16
5     10
2      6
1      4
7      1
15     1
30     1
25     1
Name: count, dtype: int64

---
18) Servicios incluidos: ninguna, una o varias de las siguientes separadas por puntos: {'luz', 'agua', 'comunidad', 'calefaccion', 'internet'}

Añadiremos una columna nueva para cada servicio, marcando 1 si está incluido y 0 en casa contrario

Como en ningun piso está incluido el internet eliminamos esa opcion

In [622]:
dataset['Servicios incluidos'].value_counts()

Servicios incluidos
agua.comunidad        35
agua.calefaccion      15
luz.comunidad         11
agua.luz.comunidad     3
luz.agua               1
luz.agua.comunidad     1
agua                   1
Name: count, dtype: int64

In [623]:
dataset['Servicios incluidos'] = dataset['Servicios incluidos'].fillna(0)

In [624]:
opciones = ['luz', 'agua', 'comunidad', 'calefaccion']

for opc in opciones:
    dataset[opc.capitalize()] = dataset['Servicios incluidos'].map(lambda x: 1 if x!=0 and opc in x.split('.') else 0)

In [625]:
dataset.head(5)

,Planta,Metros cuadrados totales,Metros cuadrados habitables,Numero habitaciones,Numero baños,Estado piso,Amueblado,Terraza,Aire acondicionado,Urbanización,Piscina en la urbanización,Ascensor,Garaje,Años antigüedad piso,Servicios incluidos,Accesibilidad,Años contrato alquiler,Precio alquiler al año,Gastos generales al año,Gastos iniciales,Beneficios totales,Precio alquiler aplicado al año,Casco viejo,Ensanche,Iturrama,Mendebaldea,Rochapea,Piso,N,S,E,W,Luz,Agua,Comunidad,Calefaccion
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,7,87.5,79.4,2,1.0,4,1,1,1,0,0,1,0,1,luz.agua,4,9,14060,1787,220,3196.814,16226.646,0,0,1,0,0,1,1,0,1,0,1,1,0,0
2,3,71.2,64.3,2,1.5,3,1,0,1,1,0,1,1,7,agua.comunidad,5,5,10525,1124,572,6227.500,13008.900,0,1,0,0,0,1,0,1,1,0,0,1,1,0
3,5,104.6,90.3,3,2.0,2,1,1,0,0,0,1,1,15,agua.luz.comunidad,3,4,18555,1512,2810,4160.112,21809.528,0,0,0,1,0,1,0,0,0,1,1,1,1,0
4,13,148.1,143.3,5,2.0,1,1,1,1,1,0,1,1,30,luz.agua.comunidad,2,10,24430,2625,7630,20183.590,29836.359,0,0,0,1,0,0,1,0,0,1,1,1,1,0
5,5,80.8,69.0,2,1.0,2,0,0,0,0,0,0,1,10,luz.comunidad,4,10,10565,1134,16440,-8424.920,12500.508,0,0,0,0,1,1,0,0,1,0,1,0,1,0


In [626]:
dataset = dataset.drop('Servicios incluidos', axis=1)

In [627]:
dataset[['Luz', 'Agua', 'Comunidad', 'Calefaccion']].value_counts()

Luz  Agua  Comunidad  Calefaccion
0    0     0          0              113
     1     1          0               35
           0          1               15
1    0     1          0               11
     1     1          0                4
0    1     0          0                1
1    1     0          0                1
Name: count, dtype: int64

---
19) Accesibilidad: int (nota del 0, en nuestro caso solo hay los siguientes valores: 3,4,5) 

In [628]:
dataset['Accesibilidad'].value_counts()

Accesibilidad
4    91
5    55
3    33
2     1
Name: count, dtype: int64

---
20) Años contrato alquiler: int

In [629]:
dataset['Años contrato alquiler'].value_counts()

Años contrato alquiler
5     24
10    20
9     19
11    19
7     18
8     17
6     13
4      9
12     8
14     6
19     6
13     5
18     4
15     4
16     3
17     3
20     2
Name: count, dtype: int64

---
---
21) Precio alquiler al año: float

In [631]:
print(dataset['Precio alquiler al año'].min())
print(dataset['Precio alquiler al año'].max())

9925.0
27570.0


---
22) Gastos generales al año: float

In [633]:
print(dataset['Gastos generales al año'].min())
print(dataset['Gastos generales al año'].max())

1039.0
2625.0


---
23) Gastos iniciales: float

In [635]:
print(dataset['Gastos iniciales'].min())
print(dataset['Gastos iniciales'].max())

30.0
26050.0


---
24) Beneficios totales: float

In [636]:
print(dataset['Beneficios totales'].min())
print(dataset['Beneficios totales'].max())

-22074.41250000001
62355.59199999995


---
Añadimos una nueva columna que van a ser los beneficios al año que calcularemos:

Beneficios al año = Beneficios totales / años contrato

25) Beneficios al año: float

In [637]:
def beneficios_al_ano(row):
    benef = row['Beneficios totales']
    anos = row['Años contrato alquiler']
    return benef/anos

In [638]:
dataset['Beneficios al año'] = dataset.apply(beneficios_al_ano, axis=1)

---
---
GUARDAR DATASET
---
---

In [639]:
etiquetas_dataset = dataset[['Precio alquiler al año', 'Gastos generales al año', 'Gastos iniciales', 'Beneficios totales', 'Beneficios al año', 'Precio alquiler aplicado al año']]
dataset = dataset.drop(['Precio alquiler al año', 'Gastos generales al año', 'Gastos iniciales', 'Beneficios totales', 'Beneficios al año', 'Precio alquiler aplicado al año'], axis=1)

In [640]:
dataset = pd.concat([dataset, etiquetas_dataset], axis=1)

In [641]:
dataset.head(5)

,Planta,Metros cuadrados totales,Metros cuadrados habitables,Numero habitaciones,Numero baños,Estado piso,Amueblado,Terraza,Aire acondicionado,Urbanización,Piscina en la urbanización,Ascensor,Garaje,Años antigüedad piso,Accesibilidad,Años contrato alquiler,Casco viejo,Ensanche,Iturrama,Mendebaldea,Rochapea,Piso,N,S,E,W,Luz,Agua,Comunidad,Calefaccion,Precio alquiler al año,Gastos generales al año,Gastos iniciales,Beneficios totales,Beneficios al año,Precio alquiler aplicado al año
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,7,87.5,79.4,2,1.0,4,1,1,1,0,0,1,0,1,4,9,0,0,1,0,0,1,1,0,1,0,1,1,0,0,14060.0,1787.0,220.0,3196.814,355.201556,16226.646
2,3,71.2,64.3,2,1.5,3,1,0,1,1,0,1,1,7,5,5,0,1,0,0,0,1,0,1,1,0,0,1,1,0,10525.0,1124.0,572.0,6227.500,1245.500000,13008.900
3,5,104.6,90.3,3,2.0,2,1,1,0,0,0,1,1,15,3,4,0,0,0,1,0,1,0,0,0,1,1,1,1,0,18555.0,1512.0,2810.0,4160.112,1040.028000,21809.528
4,13,148.1,143.3,5,2.0,1,1,1,1,1,0,1,1,30,2,10,0,0,0,1,0,0,1,0,0,1,1,1,1,0,24430.0,2625.0,7630.0,20183.590,2018.359000,29836.359
5,5,80.8,69.0,2,1.0,2,0,0,0,0,0,0,1,10,4,10,0,0,0,0,1,1,0,0,1,0,1,0,1,0,10565.0,1134.0,16440.0,-8424.920,-842.492000,12500.508


In [642]:
dataset.shape

(180, 36)

In [643]:
dir_exc = '../data/pisos_datos_numericos.xlsx'
dir_csv = '../data/pisos_datos_numericos.csv'

dataset.to_csv(dir_csv, index=True)
dataset.to_excel(dir_exc, index=True)